In [4]:
# Q1) 이 경우에서는 Type 1 error가 높을까요? Type 2 Error가 높을까요? FP, FN과 관련지어 설명해주세요

# A1) FP는 False Positive, 즉 실제로는 False 이지만 True 라고 예측한 경우입니다.
# 이는 귀무가설이 맞는데 틀리다고 예측한 Type 1 error과 연관이 있습니다.
# 또, FN은 False Negative, 즉 실제로는 True 이지만 False 라고 예측한 경우입니다.
# 이는 귀무가설이 틀린데 맞다고 예측한 Type 2 error과 연관이 있습니다.
# 이 경우에서는 threshold를 낮춰 positive로 예측되는 데이터가 많아지게 만들었습니다.
# 따라서 TP와 함께 FP, 즉 Type 1 error이 증가할 것입니다.
# 반대로, TN과 함께 FN, 즉 Type 2 error은 감소할 것입니다. 


In [5]:
# Q2-1) Accuracy, Precision and Recall이 무엇인지 TP,FP,FN,TN의 식으로 나타내 주세요.
# A2-1) Accuracy(정확도)는 TP + TN /TP + FP + TN + FN 으로 전체 예측 중 True나 False를 각각 정확히 예측한 경우의 비율입니다,
# Precision(정밀도)는 TP / TP + FP 로 True라 예측한 것 중 적중한 것의 비율을 말합니다.
# Recall(재현율)는 TP / TP + FN 로 실제 True였던 예측 중 적중한 것의 비율을 말합니다.

# Q2-2) Precision and Recall에 관한 제가 든 예시가 아닌 실생활 예시 하나를 들어주세요.생각하신 예시에서 성공확률의 threshold를 높인다는 것은 무슨 의미인가요? 생각하신 예시에서는 그 threshold를 늘리는 것이 비교적 합리적일까요? 아니면 비합리적일까요? 혹은 알 수 없을까요?
# A2-2) 스팸 메일인지 아닌지를 판단하는 경우, Precision은 스팸 메일이라고 예측한 것 중 진짜 스팸 메일인 것의 비율이고, Recall은 실제 스팸 메일인 것 중 제대로 예측한 것의 비율을 나타냅니다.
# 이 경우 threshold를 높인다는 것은 보다 깐깐하게 True라고 예측한다는, 즉 스팸 메일이라고 예측되는 경우가 줄어든다는 의미입니다.
# threshold를 높이게 되면 실제 True였던 것 중 True로 예측하는 경우가 줄어드므로 Recall이 줄어듭니다. Recall과 Prediction의 trade-off 관계로 미루어보아 Prediction은 증가할 것이라 말할 수 있습니다.
# 그런데, 스팸 메일 분류의 경우 Recall보다 Prediction이 더 중요한 지표라고 생각합니다.
# FP가 FN보다 더 큰 영향을 미치기 때문입니다. 실제로는 정상적인 메일인데 이것을 스팸 메일이라고 예측하면 일상 생활에 큰 문제가 있지만, 실제 스팸 메일을 정상적인 메일이라 분류하면 조금의 불쾌함을 느끼는 정도의 문제가 있기 때문입니다.
# 따라서 Precision을 높이기 위해 threshold를 높이는 것이 합리적이고, FP 항목을 줄이는 데에 노력해야 할 것입니다.


In [7]:
# Q3) 코드 따라해보고 주석 달기!

import torch 
import torch.nn as nn 
import torchvision.datasets as dsets 
import torchvision.transforms as transforms 
from torch.autograd import Variable

# MNIST Dataset (Images and Labels) 
# MNIST 데이터 다운로드
train_dataset = dsets.MNIST(root ='./data',  
                            train = True,  
                            transform = transforms.ToTensor(), 
                            download = True) 
  
test_dataset = dsets.MNIST(root ='./data',  
                           train = False,  
                           transform = transforms.ToTensor()) 
  
    
# Hyper Parameters 
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001    


# Dataset Loader (Input Pipline)
# MNIST 데이터 저장,  batch로 쪼갬
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,  
                                           batch_size = batch_size,  
                                           shuffle = True) 
  
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,  
                                          batch_size = batch_size,  
                                          shuffle = False)

  
# Model 
# 분석할 로지스틱 모형 정의(784개 in, 10개 out)
class LogisticRegression(nn.Module): 
    def __init__(self, input_size, num_classes): 
        super(LogisticRegression, self).__init__() 
        self.linear = nn.Linear(input_size, num_classes) 
  
    def forward(self, x): 
        out = self.linear(x) 
        return out 
  
  
model = LogisticRegression(input_size, num_classes) 
  
# Loss and Optimizer 
# loss는 cross entropy로, optimizer는 stochastic gradient descent로
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate) 
  
# Training the Model 
for epoch in range(num_epochs): 
    for i, (images, labels) in enumerate(train_loader): 
        # 차원과 데이터 구조를 맞춰줌
        images = Variable(images.view(-1, 28 * 28)) 
        labels = Variable(labels) 
  
        # Forward + Backward + Optimize 
        # gradient 0으로 초기화 후 로지스틱으로 forward 진행
        # 진행 결과로 loss계산(cross entropy) 후 back propagation으로 나온 weight update
        optimizer.zero_grad() 
        outputs = model(images) 
        loss = criterion(outputs, labels) 
        loss.backward() 
        optimizer.step() 
  
        if (i + 1) % 100 == 0: 
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                  % (epoch + 1, num_epochs, i + 1, len(train_dataset) // batch_size, loss.data)) 

# Test the Model
# 아껴놓은 test data로 정확도 측정
correct = 0
total = 0
for images, labels in test_loader: 
    images = Variable(images.view(-1, 28 * 28)) 
    outputs = model(images) 
    _, predicted = torch.max(outputs.data, 1) 
    total += labels.size(0) 
    correct += (predicted == labels).sum()
    
print('Accuracy of the model on the 10000 test images: % d %%' 
      % (100 * correct / total)) 

Epoch: [ 1/ 5], Step: [ 100/ 600], Loss: 2.2082
Epoch: [ 1/ 5], Step: [ 200/ 600], Loss: 2.1205
Epoch: [ 1/ 5], Step: [ 300/ 600], Loss: 2.0429
Epoch: [ 1/ 5], Step: [ 400/ 600], Loss: 1.9502
Epoch: [ 1/ 5], Step: [ 500/ 600], Loss: 1.8685
Epoch: [ 1/ 5], Step: [ 600/ 600], Loss: 1.7641
Epoch: [ 2/ 5], Step: [ 100/ 600], Loss: 1.7125
Epoch: [ 2/ 5], Step: [ 200/ 600], Loss: 1.6783
Epoch: [ 2/ 5], Step: [ 300/ 600], Loss: 1.5982
Epoch: [ 2/ 5], Step: [ 400/ 600], Loss: 1.5706
Epoch: [ 2/ 5], Step: [ 500/ 600], Loss: 1.5190
Epoch: [ 2/ 5], Step: [ 600/ 600], Loss: 1.5485
Epoch: [ 3/ 5], Step: [ 100/ 600], Loss: 1.5358
Epoch: [ 3/ 5], Step: [ 200/ 600], Loss: 1.3781
Epoch: [ 3/ 5], Step: [ 300/ 600], Loss: 1.3433
Epoch: [ 3/ 5], Step: [ 400/ 600], Loss: 1.3226
Epoch: [ 3/ 5], Step: [ 500/ 600], Loss: 1.3200
Epoch: [ 3/ 5], Step: [ 600/ 600], Loss: 1.2290
Epoch: [ 4/ 5], Step: [ 100/ 600], Loss: 1.2771
Epoch: [ 4/ 5], Step: [ 200/ 600], Loss: 1.1860
Epoch: [ 4/ 5], Step: [ 300/ 600], Loss:

..\aten\src\ATen\native\BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


In [9]:
# Q4-1) 2-3 In[14] 코드에서 optim.SGD를 사용하지않고 코드를 짠다면 어떤 방식으로 짜야할까요? 설명해주세요.

import torch.nn.functional as F

# SGD 사용 시 코드

# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() 
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: 0.833, b: 0.942 Cost: 0.463185
Epoch  100/1000 W: 0.706, b: 0.669 Cost: 0.064433
Epoch  200/1000 W: 0.769, b: 0.526 Cost: 0.039815
Epoch  300/1000 W: 0.818, b: 0.413 Cost: 0.024604
Epoch  400/1000 W: 0.857, b: 0.325 Cost: 0.015203
Epoch  500/1000 W: 0.888, b: 0.255 Cost: 0.009395
Epoch  600/1000 W: 0.912, b: 0.201 Cost: 0.005805
Epoch  700/1000 W: 0.931, b: 0.158 Cost: 0.003587
Epoch  800/1000 W: 0.945, b: 0.124 Cost: 0.002217
Epoch  900/1000 W: 0.957, b: 0.097 Cost: 0.001370
Epoch 1000/1000 W: 0.966, b: 0.077 Cost: 0.000846


In [11]:
#SGD 미사용 시 코드

# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

# optimizer 설정
W = torch.zeros(1)
b = torch.zeros(1)
lr = 0.01

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = (x_train * W) + b
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    # SGD 대신 일일히 미분한 식을 코드로 쳐야 함
    gradient = torch.sum((W * x_train + b - y_train) * x_train)
    b_new = torch.sum((W * x_train + b - y_train ))
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))
    W -= lr * gradient
    b -= lr * b_new

Epoch    0/1000 W: 0.000, b: 0.000 Cost: 4.666667
Epoch  100/1000 W: 0.887, b: 0.257 Cost: 0.009462
Epoch  200/1000 W: 0.921, b: 0.179 Cost: 0.004594
Epoch  300/1000 W: 0.945, b: 0.125 Cost: 0.002231
Epoch  400/1000 W: 0.962, b: 0.087 Cost: 0.001083
Epoch  500/1000 W: 0.973, b: 0.061 Cost: 0.000526
Epoch  600/1000 W: 0.981, b: 0.042 Cost: 0.000255
Epoch  700/1000 W: 0.987, b: 0.029 Cost: 0.000124
Epoch  800/1000 W: 0.991, b: 0.020 Cost: 0.000060
Epoch  900/1000 W: 0.994, b: 0.014 Cost: 0.000029
Epoch 1000/1000 W: 0.996, b: 0.010 Cost: 0.000014


In [12]:
# Q4-2) Gradient descent 코드를 구현하는 문제입니다. 
# 2-3 In[14]의 코드에서 다른 optimizer(3개 ex)adam, rmsprop, sgd에 momentum 추가 등등)를 이용하여 결과값을 비교해주세요.

# 1. SGD with momentum
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)# 보통 0.9를 사용한다고 함

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() 
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: -0.611, b: 0.114 Cost: 14.385770
Epoch  100/1000 W: 0.992, b: 0.037 Cost: 0.000443
Epoch  200/1000 W: 1.000, b: 0.001 Cost: 0.000000
Epoch  300/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  400/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  500/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  600/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  700/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  800/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  900/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch 1000/1000 W: 1.000, b: 0.000 Cost: 0.000000


In [13]:
# 2. NAG(Nesterov Accelerated Gradient)
# momentum 먼저, 그 다음 gradient

# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() 
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: 0.266, b: 0.883 Cost: 1.039017
Epoch  100/1000 W: 0.976, b: 0.055 Cost: 0.000460
Epoch  200/1000 W: 0.999, b: 0.002 Cost: 0.000000
Epoch  300/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  400/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  500/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  600/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  700/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  800/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  900/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch 1000/1000 W: 1.000, b: 0.000 Cost: 0.000000


In [14]:
# 3. Adagrad
# step size를 조정해서 많이 이동하는 변수는 step 작게, 조금 이동하는 변수는 step 크게 변화

# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() 
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: 0.207, b: -0.285 Cost: 4.043956
Epoch  100/1000 W: 0.373, b: -0.119 Cost: 2.155156
Epoch  200/1000 W: 0.443, b: -0.049 Cost: 1.561658
Epoch  300/1000 W: 0.494, b: 0.001 Cost: 1.194696
Epoch  400/1000 W: 0.534, b: 0.041 Cost: 0.939014
Epoch  500/1000 W: 0.568, b: 0.074 Cost: 0.750476
Epoch  600/1000 W: 0.597, b: 0.102 Cost: 0.606754
Epoch  700/1000 W: 0.621, b: 0.126 Cost: 0.494798
Epoch  800/1000 W: 0.643, b: 0.147 Cost: 0.406261
Epoch  900/1000 W: 0.663, b: 0.166 Cost: 0.335472
Epoch 1000/1000 W: 0.680, b: 0.182 Cost: 0.278406
